In [1]:
import gradio as gr
import os
from groq import Groq
import json
import numpy as np
import duckdb
import sqlparse
import pandas as pd
import re
from collections import Counter
from T2S import handle_question

/Users/yoohyra/Desktop/Text2SQL/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def tokenize_sql(query):
    """
    This function tokenizes the SQL query by splitting it into individual tokens.
    
    Parameters:
    query (str): The SQL query to tokenize.
    
    Returns:
    list: A list of tokens.
    """
    # Split the query into tokens
    return re.findall(r'\w+\.\w+|\*|\w+|[^\s\w]', query)

In [3]:
def clean_sql_query(query):
    """
    This function removes special characters like \n, backticks, and converts the query to lowercase.
    
    Parameters:
    query (str): The SQL query to clean.
    
    Returns:
    str: The cleaned SQL query.
    """
    # Remove special characters and convert to lowercase
    query = query.replace('\n', ' ').replace('```sql', '').replace('```', '')
    query = query.replace('\\n', ' ').strip()
    return query.lower()

In [4]:
def calculate_similarity_by_tokens(tokens1, tokens2):
    """
    This function calculates the similarity percentage between two lists of tokens.
    
    Parameters:
    tokens1 (list): The tokens of the first SQL query.
    tokens2 (list): The tokens of the second SQL query.
    
    Returns:
    float: The similarity percentage.
    """
    print("Predicted Query: ", tokens1)
    print("Test Query: ", tokens2)
    
    if not tokens1 or not tokens2:
        return 0.0
    
    common_tokens = set(tokens1) & set(tokens2)
    total_tokens = set(tokens1) | set(tokens2)
    
    return (len(common_tokens) / len(total_tokens)) * 100

In [5]:
def evaluate_accuracy(test_file, model, max_num_reflections, files):
    test_queries = pd.read_csv(test_file)
    accuracies = []

    for idx, row in test_queries.iterrows():
        user_question = row['question']
        expected_sql = row['query']  # Adjusted to match the correct column name

        # Generate SQL from LLM
        llm_sql, _, _ = handle_question(user_question, model, max_num_reflections, files)
        
        # Clean and tokenize the SQL queries
        cleaned_llm_sql = clean_sql_query(llm_sql)
        cleaned_expected_sql = clean_sql_query(expected_sql)

        tokens1 = tokenize_sql(cleaned_llm_sql)
        tokens2 = tokenize_sql(cleaned_expected_sql)
        
        # Calculate similarity
        similarity = calculate_similarity_by_tokens(tokens1, tokens2)
        accuracies.append(similarity)

        print(f"Question {idx+1}: Similarity = {similarity:.2f}%")

    return f"Accuracy calculated by tokens: ", sum(accuracies) / len(accuracies)

In [6]:
# Evaluate accuracy using the test queries
evaluate_accuracy('/Users/yoohyra/Desktop/Text2SQL/TestData/testQueries.csv', 'gemma2-9b-it', 10, None)

Predicted Query:  ['select', 'count', '(', '*', ')', 'from', 'artist']
Test Query:  ['select', 'count', '(', '*', ')', 'from', 'artist']
Question 1: Similarity = 100.00%
Predicted Query:  ['select', 'avg', '(', 'total', ')', 'from', 'invoice', 'i', 'join', 'customer', 'c', 'on', 'i.customerid', '=', 'c.customerid', 'where', 'c.country', '=', "'", 'usa', "'", 'and', 'c.fax', 'is', 'null', 'and', 'i.invoicedate', 'between', "'", '2003', '-', '01', '-', '01', "'", 'and', "'", '2010', '-', '12', '-', '31', "'"]
Test Query:  ['select', 'avg', '(', 'total', ')', 'from', 'invoice', 'i', 'join', 'customer', 'c', 'on', 'i.customerid', '=', 'c.customerid', 'where', 'c.country', '=', "'", 'usa', "'", 'and', 'i.invoicedate', 'between', "'", '2003', '-', '01', '-', '01', "'", 'and', "'", '2010', '-', '01', '-', '01', "'", 'and', 'c.fax', 'is', 'null']
Question 2: Similarity = 93.55%
Predicted Query:  ['select', 'p.firstname', ',', 'p.lastname', 'from', 'customer', 'p', 'where', 'p.country', '!', '=

('Accuracy calculated by tokens: ', 66.99434847107638)

In [7]:
def calculate_similarity_manhattan(tokens1, tokens2):
    """
    This function calculates the similarity percentage between 2 SQL queries by using Manhattan Distance.

    Parameters:
    query1 (str): The first SQL query.
    query2 (str): The second SQL query.

    Returns:
    float: The similarity percentage between 2 SQL queries.
    """

    print("Predicted Query: ", tokens1)
    print("Test Query: ", tokens2)

    # Tạo từ điển đếm số lượng từ cho mỗi câu SQL
    counter1 = Counter(tokens1)
    counter2 = Counter(tokens2)

    # Lấy tập hợp tất cả các từ trong cả hai câu
    all_tokens = set(counter1.keys()).union(set(counter2.keys()))

    # Tính toán khoảng cách Manhattan
    distance = sum(abs(counter1.get(token, 0) - counter2.get(token, 0)) for token in all_tokens)

    # Tính tổng số lượng từ trong cả hai câu
    total_tokens = sum(counter1.values()) + sum(counter2.values())

    # Tính phần trăm độ tương đồng
    similarity = 1 - (distance / total_tokens)

    return similarity * 100


In [8]:
def evaluate_accuracy_manhattan(test_file, model, max_num_reflections, files):
    test_queries = pd.read_csv(test_file)
    accuracies = []

    for idx, row in test_queries.iterrows():
        user_question = row['question']
        expected_sql = row['query']  # Adjusted to match the correct column name

        # Generate SQL from LLM
        llm_sql, _, _ = handle_question(user_question, model, max_num_reflections, files)
        
        # Clean and tokenize the SQL queries
        cleaned_llm_sql = clean_sql_query(llm_sql)
        cleaned_expected_sql = clean_sql_query(expected_sql)

        tokens1 = tokenize_sql(cleaned_llm_sql)
        tokens2 = tokenize_sql(cleaned_expected_sql)
        
        # Calculate similarity
        similarity = calculate_similarity_manhattan(tokens1, tokens2)
        # # Calculate similarity
        # similarity = calculate_similarity_manhattan(cleaned_llm_sql, cleaned_expected_sql)
        accuracies.append(similarity)

        print(f"Question {idx+1}: Similarity = {similarity:.2f}%")

    # Return average accuracy across all questions
    return f"Accuracy calculated by manhattan distance: ", (sum(accuracies) / len(accuracies))

In [9]:
# Evaluate accuracy using the test queries
evaluate_accuracy_manhattan('/Users/yoohyra/Desktop/Text2SQL/TestData/testQueries.csv', 'gemma2-9b-it', 10, None)

Predicted Query:  ['select', 'count', '(', '*', ')', 'from', 'artist']
Test Query:  ['select', 'count', '(', '*', ')', 'from', 'artist']
Question 1: Similarity = 100.00%
Predicted Query:  ['select', 'avg', '(', 't.total', ')', 'from', 'invoice', 'as', 't', 'join', 'customer', 'as', 'c', 'on', 't.customerid', '=', 'c.customerid', 'where', 'c.country', '=', "'", 'usa', "'", 'and', 'c.fax', 'is', 'null', 'and', 't.invoicedate', 'between', "'", '2003', '-', '01', '-', '01', "'", 'and', "'", '2010', '-', '01', '-', '01', "'"]
Test Query:  ['select', 'avg', '(', 'total', ')', 'from', 'invoice', 'i', 'join', 'customer', 'c', 'on', 'i.customerid', '=', 'c.customerid', 'where', 'c.country', '=', "'", 'usa', "'", 'and', 'i.invoicedate', 'between', "'", '2003', '-', '01', '-', '01', "'", 'and', "'", '2010', '-', '01', '-', '01', "'", 'and', 'c.fax', 'is', 'null']
Question 2: Similarity = 88.64%
Predicted Query:  ['select', '*', 'from', 'customer', 'where', 'country', '!', '=', "'", 'usa', "'"]
Te

('Accuracy calculated by manhattan distance: ', 74.65998560689772)